In [120]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [4]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [6]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [7]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [83]:
train_df.shape

(35040, 13)

In [9]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [71]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [72]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [189]:
K = 12
n_hours = 24*3

In [230]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08,)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=128,
    trainer=trainer,
)

model = estimator.train(train_ds)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:43<00:00,  3.43s/it, epoch=76/1000, avg_epoch_loss=4.02]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:53<00:00,  3.53s/it, epoch=152/1000, avg_epoch_loss=3.94]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [07:07<00:00,  4.27s/it, epoch=228/1000, avg_epoch_loss=3.93]


 61%|██████████████████████████████████████████████████████████████████████████████████▎                                                    | 61/100 [04:33<02:54,  4.48s/it, epoch=250/1000, avg_epoch_loss=3.93]
/lclhome/jshi008/miniconda3/lib/python3.9/site-packages/gluonts/mx/trainer/_base.py:474: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(


#### predicted y

In [231]:
# forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
# forecasts = list(forecast_it)
# tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [232]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [233]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939]
(12,)
------
1
[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878]
(24,)
------
2
[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878  96.48500061
 103.60500336 112.47499847 120.83000183 130.24000549 139.11999512
 148.07000732 160.19999695 171.63499451 181.75       189.89500427
 196.94000244]
(36,)
------
3


[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878  96.48500061
 103.60500336 112.47499847 120.83000183 130.24000549 139.11999512
 148.07000732 160.19999695 171.63499451 181.75       189.89500427
 196.94000244  93.65000153 102.29000092 110.06999969 119.60500336
 127.12000275 135.88999939 146.96499634 155.3999939  167.66000366
 174.3500061  182.08999634 183.42999268  81.36000061  90.29000092
  98.70999908 105.27999878 113.84999847 122.03500366 128.65499878
 136.44999695 142.66499329 148.88999939 152.56500244 158.64500427
  82.70999908  92.75        99.83999634 109.89499664 117.70500183
 123.38500214 131.39500427 137.05000305 142.6000061  145.32000732
 152.13499451 149.11999512  81.09999847  88.56500244  97.63999939
 105.360000

[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878  96.48500061
 103.60500336 112.47499847 120.83000183 130.24000549 139.11999512
 148.07000732 160.19999695 171.63499451 181.75       189.89500427
 196.94000244  93.65000153 102.29000092 110.06999969 119.60500336
 127.12000275 135.88999939 146.96499634 155.3999939  167.66000366
 174.3500061  182.08999634 183.42999268  81.36000061  90.29000092
  98.70999908 105.27999878 113.84999847 122.03500366 128.65499878
 136.44999695 142.66499329 148.88999939 152.56500244 158.64500427
  82.70999908  92.75        99.83999634 109.89499664 117.70500183
 123.38500214 131.39500427 137.05000305 142.6000061  145.32000732
 152.13499451 149.11999512  81.09999847  88.56500244  97.63999939
 105.360000

[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878  96.48500061
 103.60500336 112.47499847 120.83000183 130.24000549 139.11999512
 148.07000732 160.19999695 171.63499451 181.75       189.89500427
 196.94000244  93.65000153 102.29000092 110.06999969 119.60500336
 127.12000275 135.88999939 146.96499634 155.3999939  167.66000366
 174.3500061  182.08999634 183.42999268  81.36000061  90.29000092
  98.70999908 105.27999878 113.84999847 122.03500366 128.65499878
 136.44999695 142.66499329 148.88999939 152.56500244 158.64500427
  82.70999908  92.75        99.83999634 109.89499664 117.70500183
 123.38500214 131.39500427 137.05000305 142.6000061  145.32000732
 152.13499451 149.11999512  81.09999847  88.56500244  97.63999939
 105.360000

[122.86499786 135.25999451 147.21499634 157.13499451 171.19500732
 178.58000183 190.58999634 202.86000061 208.16499329 215.90499878
 219.27000427 228.13999939  94.81999969 108.34999847 114.44499969
 122.31500244 129.36000061 140.75       155.30999756 163.3999939
 176.33999634 184.83999634 198.99000549 202.52999878  96.48500061
 103.60500336 112.47499847 120.83000183 130.24000549 139.11999512
 148.07000732 160.19999695 171.63499451 181.75       189.89500427
 196.94000244  93.65000153 102.29000092 110.06999969 119.60500336
 127.12000275 135.88999939 146.96499634 155.3999939  167.66000366
 174.3500061  182.08999634 183.42999268  81.36000061  90.29000092
  98.70999908 105.27999878 113.84999847 122.03500366 128.65499878
 136.44999695 142.66499329 148.88999939 152.56500244 158.64500427
  82.70999908  92.75        99.83999634 109.89499664 117.70500183
 123.38500214 131.39500427 137.05000305 142.6000061  145.32000732
 152.13499451 149.11999512  81.09999847  88.56500244  97.63999939
 105.360000

...

[122.86499786 135.25999451 147.21499634 ...  22.39999962  23.37000084
  22.93499947]
(99960,)
------
8330
[122.86499786 135.25999451 147.21499634 ...  20.45499992  20.28499985
  20.18499947]
(99972,)
------
8331
[122.86499786 135.25999451 147.21499634 ...  21.73500061  22.25499916
  22.71999931]
(99984,)
------
8332
[122.86499786 135.25999451 147.21499634 ...  32.06999969  32.29999924
  33.31499863]
(99996,)
------
8333
[122.86499786 135.25999451 147.21499634 ...  48.79499817  43.08000183
  41.90000153]
(100008,)
------
8334
[122.86499786 135.25999451 147.21499634 ...  38.15499878  36.94499969
  36.09500122]
(100020,)
------
8335
[122.86499786 135.25999451 147.21499634 ...  34.44499969  34.79999924
  37.36999893]
(100032,)
------
8336
[122.86499786 135.25999451 147.21499634 ...  37.43000031  41.47499847
  42.54999924]
(100044,)
------
8337
[122.86499786 135.25999451 147.21499634 ...  51.42499924  55.56999969
  63.00999832]
(100056,)
------
8338
[122.86499786 135.25999451 147.21499634 .

[122.86499786 135.25999451 147.21499634 ...  12.88000011  12.49499989
  12.14999962]
(100884,)
------
8407
[122.86499786 135.25999451 147.21499634 ...  14.02000046  13.96000004
  13.92500019]
(100896,)
------
8408
[122.86499786 135.25999451 147.21499634 ...  11.73999977  12.57499981
  12.44499969]
(100908,)
------
8409
[122.86499786 135.25999451 147.21499634 ...   4.73999977   5.7750001
   6.16499996]
(100920,)
------
8410
[122.86499786 135.25999451 147.21499634 ...   4.625        4.875
   4.81500006]
(100932,)
------
8411
[122.86499786 135.25999451 147.21499634 ...   5.54500008   4.28000021
   5.15999985]
(100944,)
------
8412
[122.86499786 135.25999451 147.21499634 ...   3.79500008   4.55499983
   4.63999987]
(100956,)
------
8413
[122.86499786 135.25999451 147.21499634 ...   3.29500008   5.21000004
   5.10500002]
(100968,)
------
8414
[122.86499786 135.25999451 147.21499634 ...   4.32499981   4.53999996
   4.15999985]
(100980,)
------
8415
[122.86499786 135.25999451 147.21499634 ...

[122.86499786 135.25999451 147.21499634 ...  52.56499863  50.04499817
  50.27999878]
(101820,)
------
8485
[122.86499786 135.25999451 147.21499634 ...  59.61500168  61.86500168
  61.24499893]
(101832,)
------
8486
[122.86499786 135.25999451 147.21499634 ...  70.73500061  67.85500336
  64.625     ]
(101844,)
------
8487
[122.86499786 135.25999451 147.21499634 ...  77.81999969  70.71499634
  61.59999847]
(101856,)
------
8488
[122.86499786 135.25999451 147.21499634 ...  85.94999695  71.13500214
  68.90000153]
(101868,)
------
8489
[122.86499786 135.25999451 147.21499634 ...  60.60499954  54.89500046
  48.79499817]
(101880,)
------
8490
[122.86499786 135.25999451 147.21499634 ...  53.97000122  46.84999847
  45.68500137]
(101892,)
------
8491
[122.86499786 135.25999451 147.21499634 ...  86.44000244  80.22499847
  80.44000244]
(101904,)
------
8492
[122.86499786 135.25999451 147.21499634 ...  71.17500305  63.83499908
  62.59500122]
(101916,)
------
8493
[122.86499786 135.25999451 147.214996

[122.86499786 135.25999451 147.21499634 ... 184.30499268 167.71000671
 159.39500427]
(102756,)
------
8563
[122.86499786 135.25999451 147.21499634 ... 190.80499268 177.49000549
 172.6499939 ]
(102768,)
------
8564
[122.86499786 135.25999451 147.21499634 ... 198.00500488 189.82000732
 180.69000244]
(102780,)
------
8565
[122.86499786 135.25999451 147.21499634 ... 178.3500061  173.32499695
 167.84500122]
(102792,)
------
8566
[122.86499786 135.25999451 147.21499634 ... 184.73500061 178.66999817
 164.20500183]
(102804,)
------
8567
[122.86499786 135.25999451 147.21499634 ... 171.         158.0249939
 151.38000488]
(102816,)
------
8568
[122.86499786 135.25999451 147.21499634 ... 141.44000244 125.82499695
 106.54499817]
(102828,)
------
8569
[122.86499786 135.25999451 147.21499634 ... 108.10500336  91.58499908
  82.23999786]
(102840,)
------
8570
[122.86499786 135.25999451 147.21499634 ... 106.29499817  88.30999756
  83.23999786]
(102852,)
------
8571
[122.86499786 135.25999451 147.2149963

[122.86499786 135.25999451 147.21499634 ...  79.15499878  76.63500214
  70.77999878]
(103680,)
------
8640
[122.86499786 135.25999451 147.21499634 ...  81.66999817  76.02500153
  75.09500122]
(103692,)
------
8641
[122.86499786 135.25999451 147.21499634 ...  93.66500092  90.35500336
  92.26999664]
(103704,)
------
8642
[122.86499786 135.25999451 147.21499634 ...  74.51499939  79.62999725
  86.75      ]
(103716,)
------
8643
[122.86499786 135.25999451 147.21499634 ...  74.44499969  79.95500183
  86.12999725]
(103728,)
------
8644
[122.86499786 135.25999451 147.21499634 ...  57.50500107  64.41999817
  66.60500336]
(103740,)
------
8645
[122.86499786 135.25999451 147.21499634 ...  59.52000046  64.48000336
  66.55500031]
(103752,)
------
8646
[122.86499786 135.25999451 147.21499634 ...  50.14500046  56.02500153
  58.13499832]
(103764,)
------
8647
[122.86499786 135.25999451 147.21499634 ...  43.11500168  45.22999954
  51.16999817]
(103776,)
------
8648
[122.86499786 135.25999451 147.214996

In [234]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(8677, 12)

In [235]:
test_ds_predict_q50_supervised[0:5]

array([[122.86499786, 135.25999451, 147.21499634, 157.13499451,
        171.19500732, 178.58000183, 190.58999634, 202.86000061,
        208.16499329, 215.90499878, 219.27000427, 228.13999939],
       [ 94.81999969, 108.34999847, 114.44499969, 122.31500244,
        129.36000061, 140.75      , 155.30999756, 163.3999939 ,
        176.33999634, 184.83999634, 198.99000549, 202.52999878],
       [ 96.48500061, 103.60500336, 112.47499847, 120.83000183,
        130.24000549, 139.11999512, 148.07000732, 160.19999695,
        171.63499451, 181.75      , 189.89500427, 196.94000244],
       [ 93.65000153, 102.29000092, 110.06999969, 119.60500336,
        127.12000275, 135.88999939, 146.96499634, 155.3999939 ,
        167.66000366, 174.3500061 , 182.08999634, 183.42999268],
       [ 81.36000061,  90.29000092,  98.70999908, 105.27999878,
        113.84999847, 122.03500366, 128.65499878, 136.44999695,
        142.66499329, 148.88999939, 152.56500244, 158.64500427]])

#### actual y

In [236]:
from helper import series_to_supervised, stage_series_to_supervised

In [237]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [238]:
test_df_supervised.iloc[:, -K:].shape

(8677, 12)

In [239]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [240]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5),var1(t+6),var1(t+7),var1(t+8),var1(t+9),var1(t+10),var1(t+11)
35112,85.0,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0
35113,86.0,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0
35114,89.0,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0
35115,77.0,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0
35116,77.0,75.0,80.0,86.0,95.0,101.0,132.0,153.0,173.0,178.0,176.0,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43784,8.0,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0
43785,8.0,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0
43786,7.0,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0
43787,12.0,17.0,11.0,9.0,11.0,8.0,9.0,10.0,8.0,10.0,10.0,8.0


#### performance on test set

In [241]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 29.958
RMSE = 50.341
